In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.webdriver import WebDriver
from typing import Any
import json
from webdriver_manager.chrome import ChromeDriverManager 

In [4]:
class Driver(WebDriver):
    """Webdriver with all necessary options set."""

    log_list: list[dict[str, Any]] = []

    def __init__(self) -> None:
        """Instance driver will all necessary options."""
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument("--ignore-certificate-errors")
        chrome_options.add_argument("--ignore-ssl-errors=yes")
        chrome_options.set_capability("goog:loggingPrefs", {"performance": "ALL"})
        super().__init__(options=chrome_options, service=Service(ChromeDriverManager().install()))

    def get_response(self, request_id: str) -> dict[str, Any] | None:
        """Get response of a request through Chrome DevTools Protocol.

        Args:
            request_id (str): Id of the request.

        Returns:
            dict[str, Any] | None: Response of the request.
        """
        if not request_id:
            return None
        resource = "/session/%s/chromium/send_command_and_get_result" % self.session_id
        url = self.command_executor._url + resource
        body = json.dumps({"cmd": "Network.getResponseBody", "params": {"requestId": request_id}})
        response = self.command_executor._request("POST", url, body)
        if not response:
            print(f"No response found for request {request_id}")
            return None
        response = response.get("value")
        if (
            not response
            or "No resource with given identifier found" in response
            or "No data found for resource with given identifier" in response
        ):
            print(f"No response found for request {request_id}")
            return None

        return response

    def get_performance_logs(self) -> list[dict[str, Any]]:
        """Get performance logs."""
        logs_raw = self.get_log("performance")
        logs = [json.loads(lr["message"])["message"] for lr in logs_raw]
        self.log_list.extend(logs)
        return self.log_list

    def get_requests(self) -> list[dict[str, Any]]:
        """Get all requests made by the browser.

        Returns:
            list[dict[str, Any]]: List of requests made by the browser.
        """
        logs = self.get_performance_logs()
        requests = [log.get("params", {}) for log in logs if "Network.requestWillBeSent" in log["method"]]
        return requests

    def check_response_status(self, request_id: str) -> int | None:
        """Check status of a request.

        Args:
            request_id (str): Id of the request.

        Returns:
            int | None: Status of the request.
        """
        logs = self.get_performance_logs()
        logs = [
            log
            for log in logs
            if "Network.responseReceived" in log["method"]
            and "params" in log
            and "requestId" in log["params"]
            and log["params"]["requestId"] == request_id
        ]
        if not logs:
            print(f"No response found in logs for request {request_id}")
            return None
        print(f"Found {len(logs)} responses for request {request_id}")
        for log in logs:
            params = log.get("params", {})
            if not params:
                print(f"No params found in logs for request {request_id}")
                continue
            res = params.get("response", {})
            if not res:
                print(f"No response found in logs for request {request_id}")
                continue
            status = res.get("status", None)
            return status
        print(f"No response found in logs for request {request_id}")
        return None

    def clear_performance_logs(self) -> None:
        """Clear performance logs."""
        self.log_list = []



In [5]:
driver = Driver()
url = "https://www.worten.pt/"
driver.get(url)
driver.implicitly_wait(5) # Not the best approach


In [6]:
# So we got a cookie pop-up, let's accept it to get it out of the way
cookies_modal = driver.find_element(By.CSS_SELECTOR,"div.modal__content")
accept_button = cookies_modal.find_element(By.XPATH,".//button[span[normalize-space()='Aceitar cookies']]").click()

In [7]:
to_search = "iphone 16"
search_textbox = driver.find_element(By.ID, 'search')
search_textbox.send_keys(to_search)
from selenium.webdriver.common.keys import Keys
search_textbox.send_keys(Keys.RETURN)

driver.implicitly_wait(10) # Not the best approach


In [8]:
def get_api_response(request_id: str) -> dict[str, Any]:
    """Get response from the network interceptor."""
    response = driver.get_response(request_id)
    if not response:
        return {}
    body: str | None = ""
    try:
        if type(response) is not dict:  # noqa: E721
            print(f"Response is not a dict!!: {response}")
            return {}
        body = response.get("body", None)
    except Exception as e:
        print(f"Response: {response}")
        raise e

    if not body:
        return {}
    
    encoded = body.encode("utf-8")
    body_json = json.loads(encoded)
    data = body_json.get("data")
    if data is None:
        print(f"`data` is missing from the response: {response}")
        return {}
    return data

In [12]:
requests = driver.get_requests()
requests = [
    request
    for request in requests
    if "request" in request
    and "url" in request["request"]
    and "wOperationName=searchProducts" in request["request"]["url"]
    and "requestId" in request
]

In [13]:
requests

[{'documentURL': 'https://www.worten.pt/search?query=iphone+16',
  'frameId': '7E7BA4BC26F55F01C87AEF77171453EC',
  'hasUserGesture': True,
  'initiator': {'stack': {'callFrames': [{'columnNumber': 2759,
      'functionName': '',
      'lineNumber': 217,
      'scriptId': '12',
      'url': 'https://www.worten.pt/assetsV4/hMpQHLUL.js'},
     {'columnNumber': 4722,
      'functionName': '',
      'lineNumber': 200,
      'scriptId': '12',
      'url': 'https://www.worten.pt/assetsV4/hMpQHLUL.js'},
     {'columnNumber': 2485,
      'functionName': 'e',
      'lineNumber': 195,
      'scriptId': '12',
      'url': 'https://www.worten.pt/assetsV4/hMpQHLUL.js'},
     {'columnNumber': 3432,
      'functionName': 't.subscribe',
      'lineNumber': 195,
      'scriptId': '12',
      'url': 'https://www.worten.pt/assetsV4/hMpQHLUL.js'},
     {'columnNumber': 95890,
      'functionName': '',
      'lineNumber': 200,
      'scriptId': '12',
      'url': 'https://www.worten.pt/assetsV4/hMpQHLUL.js

In [14]:
responses = []
for request in requests:
    request_id = request["requestId"]
    response = get_api_response(request_id)
    responses.append(response)

In [27]:
products = responses[0]['searchProducts']['hits']
len(products)
products[0]['product']
products_parsed = []
for product in products:
    product = product['product']
    name = product['description']
    price = product['textProperties']['price']
    products_parsed.append({'name': name, 'price': price})
    
products_parsed

[{'name': 'iPhone 16 Pro Max 256GB Desert Titanium', 'price': '1499'},
 {'name': 'iPhone 16 Pro Max 256GB Black Titanium', 'price': '1499'},
 {'name': 'iPhone 16 Pro 128GB Black Titanium', 'price': '1249'},
 {'name': 'iPhone 16 Pro 256GB Black Titanium', 'price': '1379'},
 {'name': 'iPhone 16 128GB Pink', 'price': '989.99'},
 {'name': 'iPhone 16 Pro Max 256GB White Titanium', 'price': '1499'},
 {'name': 'iPhone 16 Pro 256GB Natural Titanium', 'price': '1379'},
 {'name': 'iPhone 16 Pro 128GB Desert Titanium', 'price': '1249'},
 {'name': 'iPhone 16 Pro 128GB Natural Titanium', 'price': '1249'},
 {'name': 'iPhone 16 Pro Max 256GB Natural Titanium', 'price': '1499'},
 {'name': 'iPhone 16 Pro Max 512GB Black Titanium', 'price': '1749'},
 {'name': 'iPhone 16 Pro 256GB White Titanium', 'price': '1379'},
 {'name': 'iPhone 16 Pro 128GB White Titanium', 'price': '1249'},
 {'name': 'iPhone 16 Pro Max 512GB Natural Titanium', 'price': '1749'},
 {'name': 'iPhone 16 128GB Ultramarine', 'price': '989

In [28]:
driver.quit()